In [2]:
import pandas as pd
import numpy  as np
from sklearn import datasets

X, y = datasets.make_moons(10000, noise=0.20)

# Can't believe I'm doing it this way, but join doesn't work
# on numpy strings and I'm on a plane unable to lookup the
# right way to join a column to a matrix and output as CSV.
for x_i,y_i in zip(X,y):
    output = ''
    output += str(y_i)
    for j in range(0,len(x_i)):
        output += ','
        output += str(x_i[j])
    # print output

In [3]:
NObs =X.shape[0]

traindata=X[0:NObs/2,:]
validdata=X[NObs/2+1:NObs*3/4,:]
testdata=X[NObs*3/4+1:,:]

train_labels=pd.DataFrame(y[0:NObs/2])
valid_labels=pd.DataFrame(y[NObs/2+1:NObs*3/4])

test_labels=pd.DataFrame(y[NObs*3/4+1:])

train_labels.columns = ['label']
valid_labels.columns = ['label']
test_labels.columns = ['label']

In [4]:
# prepare data for TensorFlow

num_var = 2
num_labels = 2 # 39 categories

train_dataset = np.float32(np.array(traindata))
N_obs_train=len(train_dataset)

valid_dataset = np.float32(np.array(validdata))

test_dataset = np.float32(np.array(testdata))

train_labels= np.array(pd.get_dummies(train_labels['label'].astype('category').cat.rename_categories(np.arange(0,num_labels)) ))
valid_labels= np.array(pd.get_dummies(valid_labels['label'].astype('category').cat.rename_categories(np.arange(0,num_labels)) ))
test_labels= np.array(pd.get_dummies ( test_labels['label'].astype('category').cat.rename_categories(np.arange(0,num_labels)) ))

print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape)

test_dataset.dtype

('Training set', (5000, 2), (5000, 2))
('Validation set', (2499, 2), (2499, 2))
('Test set', (2499, 2))


dtype('float32')

In [5]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

In [6]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

def baseline_logloss(hotmatrix): #input is a hot-encoded sparse matrix, one 1 only per row, rest is 0
    nobs = len(hotmatrix)
    temp = hotmatrix.sum(axis=0)/nobs
    return(-sum(np.multiply(temp,np.log(temp))))

In [16]:
# Adding regularization to the 1 hidden layer network
graph1 = tf.Graph()
batch_size = 128

epoch_size  = N_obs_train / batch_size
num_steps= 5 * epoch_size

import datetime
startTime = datetime.datetime.now() 

def define_and_run_batch(beta):
    
    num_RELU = 10
    
    with graph1.as_default():
      
      keep_prob = tf.placeholder(tf.float32)
      
      # Input data. For the training data, we use a placeholder that will be fed
      # at run time with a training minibatch.
      tf_train_dataset = tf.placeholder(tf.float32,
                                        shape=(batch_size, num_var)) # remplace batch_size par None
      tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
      tf_valid_dataset = tf.constant(valid_dataset)
    
      # in Kaggle competitions, we don't know the true labels of the test set
      tf_test_dataset = tf.constant(test_dataset)
  
      # Variables.
      weights_RELU = tf.Variable(
        tf.truncated_normal([num_var, num_RELU],stddev=0.1))
        
      # print(tf.shape(weights_RELU) )   
      #biases_RELU = tf.Variable(tf.zeros([num_RELU]))
      biases_RELU = tf.Variable(tf.random_normal([num_RELU],stddev=0.1))  
        
      weights_layer1 = tf.Variable(
        tf.truncated_normal([num_RELU, num_labels],stddev=0.1))
      #biases_layer1 = tf.Variable(tf.zeros([num_labels]))
      biases_layer1 = tf.Variable(tf.random_normal([num_labels],stddev=0.1))
  
      # Training computation.
      logits_RELU = tf.matmul(tf_train_dataset, weights_RELU) + biases_RELU
      RELU_vec = tf.nn.relu(logits_RELU)
      RELU_vec_drop = tf.nn.dropout(RELU_vec, keep_prob)
    
      logits_layer = tf.matmul(RELU_vec_drop, weights_layer1) + biases_layer1                  
      # loss = tf.reduce_mean(
      #        tf.nn.softmax_cross_entropy_with_logits(logits_layer, tf_train_labels))
      cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits_layer, tf_train_labels,name="cross_entropy")
      l2reg = tf.reduce_sum(tf.square(weights_RELU))+tf.reduce_sum(tf.square(weights_layer1))
      
      loss = tf.reduce_mean(cross_entropy+beta*l2reg)
      logloss = tf.reduce_mean(cross_entropy)
    
  # Optimizer.
      optimizer = tf.train.GradientDescentOptimizer(0.3).minimize(loss)
  
      # Predictions for the training, validation, and test data.
      train_prediction = tf.nn.softmax(logits_layer)
      
      valid_prediction = tf.nn.softmax(
        tf.matmul(tf.nn.relu((tf.matmul(tf_valid_dataset, weights_RELU) + biases_RELU)),weights_layer1)+biases_layer1)
        
      # in Kaggle competitions, we don't know the true labels of the test set     
      test_prediction =tf.nn.softmax(
        tf.matmul(tf.nn.relu((tf.matmul(tf_test_dataset, weights_RELU) + biases_RELU)),weights_layer1)+biases_layer1)
                         
    with tf.Session(graph=graph1) as session:
 
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch. 
        batch_data = train_dataset[offset:(offset + batch_size), :]
        
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,
                    keep_prob:1.0}
        
        #
        if (step==num_steps-1):
            _, l, predictions_train,predictions_test,predictions_valid = session.run(
              [optimizer, logloss,train_prediction,test_prediction,valid_prediction], feed_dict=feed_dict)
        else:
            _, l, predictions_train = session.run(
              [optimizer, logloss,train_prediction], feed_dict=feed_dict)
            
        if (step % 50 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions_train, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      
      # test_acc = accuracy(test_prediction.eval(), test_labels)
      # print("Test accuracy: %.1f%%" % test_acc)

      
        
    x = datetime.datetime.now() - startTime
    print(x)
    return(predictions_test,predictions_valid,predictions_train)
    
    
RES,RES_v,RES_tr=define_and_run_batch(0.001)
N_obs_valid = validdata.shape[0]
N_obs_test  =  testdata.shape[0]
logloss_valid=-sum(sum(np.multiply(valid_labels,np.log(RES_v))))/N_obs_valid
logloss_test =-sum(sum(np.multiply( test_labels,np.log(RES  ))))/N_obs_test
print("Validation Logloss =%s" % logloss_valid)
print("Test       Logloss =%s" % logloss_test)

Initialized
Minibatch loss at step 0: 0.687988
Minibatch accuracy: 56.2%
Validation accuracy: 60.7%
Minibatch loss at step 50: 0.342020
Minibatch accuracy: 84.4%
Validation accuracy: 85.7%
Minibatch loss at step 100: 0.308745
Minibatch accuracy: 86.7%
Validation accuracy: 87.3%
Minibatch loss at step 150: 0.254597
Minibatch accuracy: 87.5%
Validation accuracy: 87.3%
0:00:00.761320
Validation Logloss =0.274545674243
Test       Logloss =0.283503139714


In [26]:
# 4 layer network

graph1 = tf.Graph()

batch_size = 128

epoch_size  = N_obs_train / batch_size
num_steps= 2 * epoch_size #6501

import datetime
startTime = datetime.datetime.now() 

def define_and_run_batch_4_layer():
    
    

    import math as math

    initial_learning_rate_value = 0.5
    beta1 = 0.001
    beta2 = 0.001
    beta3 = 0.001
    beta4 = 0.001
    beta5 = 0.001

    graph2 = tf.Graph()
    with graph2.as_default():

       # Input data. For the training data, we use a placeholder that will be fed
       # at run time with a training minibatch.
       tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, num_var))
       tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
        
       tf_valid_dataset = tf.constant(valid_dataset)
       tf_test_dataset = tf.constant(test_dataset)    
    
       # learning rate decay
       global_step = tf.Variable(0)
       learning_rate = tf.train.exponential_decay(initial_learning_rate_value, global_step, 1, 0.9999)

       # Hidden layer 1
       h1_size = 32  
       weights_h1 = tf.Variable(tf.truncated_normal([num_var, h1_size], stddev=math.sqrt(2.0/(num_var))))
       biases_h1 = tf.Variable(tf.zeros([h1_size]))
       h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_h1)

       # Hidden layer 2
       h2_size = 16
       weights_h2 = tf.Variable(tf.truncated_normal([h1_size, h2_size], stddev=math.sqrt(2.0/h1_size)))
       biases_h2 = tf.Variable(tf.zeros([h2_size]))
       h2 = tf.nn.relu(tf.matmul(h1, weights_h2) + biases_h2)

       # Hidden layer 3
       h3_size = 8
       weights_h3 = tf.Variable(tf.truncated_normal([h2_size, h3_size], stddev=math.sqrt(2.0/h2_size)))
       biases_h3 = tf.Variable(tf.zeros([h3_size]))
       h3 = tf.nn.relu(tf.matmul(h2, weights_h3) + biases_h3)

       # Hidden layer 4
       h4_size = 4
       weights_h4 = tf.Variable(tf.truncated_normal([h3_size, h4_size], stddev=math.sqrt(2.0/h3_size)))
       biases_h4 = tf.Variable(tf.zeros([h4_size]))
       h4 = tf.nn.relu(tf.matmul(h3, weights_h4) + biases_h4)

       # Output layer
       weights_o = tf.Variable(tf.truncated_normal([h4_size, num_labels], stddev=math.sqrt(2.0/h4_size)))
       biases_o = tf.Variable(tf.zeros([num_labels]))
    
       # Training computation.
       logits = tf.matmul(h4, weights_o) + biases_o
       # loss = tf.reduce_mean(
       #     tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels)) + ((beta1 * tf.nn.l2_loss(weights_h1)) + (beta2 * tf.nn.l2_loss(weights_h2)) 
       #        + (beta3 * tf.nn.l2_loss(weights_h3)) + (beta4 * tf.nn.l2_loss(weights_h4)) + (beta5 * tf.nn.l2_loss(weights_o)))
       
       cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels,name="cross_entropy")
       logloss = tf.reduce_mean(cross_entropy)
    
       loss = logloss + ((beta1 * tf.nn.l2_loss(weights_h1)) + (beta2 * tf.nn.l2_loss(weights_h2)) + (beta3 * tf.nn.l2_loss(weights_h3)) 
                         + (beta4 * tf.nn.l2_loss(weights_h4)) + (beta5 * tf.nn.l2_loss(weights_o)))
       
    
       # Optimizer.
       optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)

       # Predictions for the training, validation, and test data.
       train_h1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_h1) + biases_h1)
       train_h2 = tf.nn.relu(tf.matmul(train_h1, weights_h2) + biases_h2)
       train_h3 = tf.nn.relu(tf.matmul(train_h2, weights_h3) + biases_h3)
       train_h4 = tf.nn.relu(tf.matmul(train_h3, weights_h4) + biases_h4)
       train_logits = tf.matmul(train_h4, weights_o) + biases_o
       train_prediction = tf.nn.softmax(train_logits)
  
       valid_h1 = tf.nn.relu(tf.matmul(tf_valid_dataset, weights_h1) + biases_h1)
       valid_h2 = tf.nn.relu(tf.matmul(valid_h1, weights_h2) + biases_h2)
       valid_h3 = tf.nn.relu(tf.matmul(valid_h2, weights_h3) + biases_h3)
       valid_h4 = tf.nn.relu(tf.matmul(valid_h3, weights_h4) + biases_h4)
       valid_logits = tf.matmul(valid_h4, weights_o) + biases_o
       valid_prediction = tf.nn.softmax(valid_logits)
    
       test_h1 = tf.nn.relu(tf.matmul(tf_test_dataset, weights_h1) + biases_h1)
       test_h2 = tf.nn.relu(tf.matmul(test_h1, weights_h2) + biases_h2)
       test_h3 = tf.nn.relu(tf.matmul(test_h2, weights_h3) + biases_h3)
       test_h4 = tf.nn.relu(tf.matmul(test_h3, weights_h4) + biases_h4)
       test_logits = tf.matmul(test_h4, weights_o) + biases_o
       test_prediction = tf.nn.softmax(test_logits)
  
        
   

# executing the graph     
        
        
    with tf.Session(graph=graph2) as session:
 
      tf.initialize_all_variables().run()
      print("Initialized")
      for step in range(num_steps):
        
        # Pick an offset within the training data, which has been randomized.
        # Note: we could use better randomization across epochs.
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        
        # Generate a minibatch. 
        batch_data = train_dataset[offset:(offset + batch_size), :]
        
        batch_labels = train_labels[offset:(offset + batch_size)]
        
        # Prepare a dictionary telling the session where to feed the minibatch.
        # The key of the dictionary is the placeholder node of the graph to be fed,
        # and the value is the numpy array to feed to it
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        
        #
        if (step==num_steps-1):
            _, l, predictions_train,predictions_test,predictions_valid = session.run(
              [optimizer, logloss,train_prediction,test_prediction,valid_prediction], feed_dict=feed_dict)
        else:
            _, l, predictions_train = session.run(
              [optimizer, logloss,train_prediction], feed_dict=feed_dict)
            
        if (step % 500 == 0):
          print("Minibatch loss at step %d: %f" % (step, l))
          print("Minibatch accuracy: %.1f%%" % accuracy(predictions_train, batch_labels))
          print("Validation accuracy: %.1f%%" % accuracy(
            valid_prediction.eval(), valid_labels))
      
      # test_acc = accuracy(test_prediction.eval(), test_labels)
      # print("Test accuracy: %.1f%%" % test_acc)
        
    x = datetime.datetime.now() - startTime
    print(x)
    return(predictions_test,predictions_valid)
    
    
    
RES,RES_v=define_and_run_batch_4_layer()

N_obs_valid = validdata.shape[0]
N_obs_test  =  testdata.shape[0]
logloss_valid=-sum(sum(np.multiply(valid_labels,np.log(RES_v))))/N_obs_valid
logloss_test =-sum(sum(np.multiply( test_labels,np.log(RES  ))))/N_obs_test
print("Validation Logloss =%s" % logloss_valid)
print("Test       Logloss =%s" % logloss_test)

Initialized
Minibatch loss at step 0: 0.685639
Minibatch accuracy: 49.2%
Validation accuracy: 64.1%
0:00:00.687200
Validation Logloss =0.197863387164
Test       Logloss =0.198305062552


In [27]:
print(baseline_logloss(train_labels))
print(baseline_logloss(valid_labels))

0.692859152906
0.692887029939


In [28]:
print(accuracy(RES_tr,train_labels))
print(accuracy(RES_v ,valid_labels))
print(accuracy(RES   , test_labels))

0.0
93.7575030012
93.1972789116


/Library/Frameworks/Python.framework/Versions/2.7/lib/python2.7/site-packages/ipykernel/__main__.py:3: DeprecationWarning: elementwise == comparison failed; this will raise an error in the future.
  app.launch_new_instance()


In [29]:
import matplotlib.pyplot as plt


In [30]:
test_labels

array([[ 1.,  0.],
       [ 1.,  0.],
       [ 1.,  0.],
       ..., 
       [ 1.,  0.],
       [ 0.,  1.],
       [ 0.,  1.]])

In [31]:
df=pd.DataFrame(testdata)
df.columns=['X','Y']
df['pred']=np.argmax(RES,1)
df['label']=y[NObs*3/4+1:]

def confusion (pred,label):
    if    (pred==1) and (label==1): return ('true_pos')
    elif  (pred==1) and (label==0): return ('false_pos')
    elif  (pred==0) and (label==0): return ('true_neg')
    else                          : return ('false_neg')

def myfun(row):
    return(confusion(row['pred'],row['label']))
df['class']=df.apply(myfun,axis=1)
df

,X,Y,pred,label,class
0,-0.771608,0.676851,0,0,true_neg
1,-0.672482,0.620596,0,0,true_neg
2,0.270528,0.898430,0,0,true_neg
3,0.291727,0.819991,0,0,true_neg
4,0.112167,-0.143066,1,1,true_pos
5,0.906015,-0.342094,1,1,true_pos
6,1.939017,0.153691,1,1,true_pos
7,1.845869,0.196277,1,1,true_pos
8,1.871419,-0.045443,1,1,true_pos
9,0.415073,0.170630,1,1,true_pos


In [32]:
plt.plot(df['X'],df['Y'],'ro')
plt.show()

In [33]:
grouped = df.groupby('class')

In [34]:
# Plot
fig, ax = plt.subplots()
ax.margins(0.05) # Optional, just adds 5% padding to the autoscaling
for name, group in grouped:
    ax.plot(group.X, group.Y, marker='o', linestyle='', ms=12, label=name)
ax.legend()

plt.show()